In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
from tensorflow.python.client import device_lib
import os
from tensorflow.keras.utils import image_dataset_from_directory
from matplotlib import pyplot as plt
import math
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from PIL import Image, ImageEnhance
import random
import cv2
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
import tensorflow_hub as hub
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
import warnings
import kagglehub
# tushar5harma_plant_village_dataset_updated_path = kagglehub.dataset_download('tushar5harma/plant-village-dataset-updated')

print('Data source import complete.')
vit_model = hub.load("https://tfhub.dev/sayakpaul/vit_r50_l32_fe/1")

@keras.saving.register_keras_serializable()
def vit_features(x):
    return vit_model(x)

Data source import complete.


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

MODEL_FILE = "models/plm.keras"
vit_model = hub.load("https://tfhub.dev/sayakpaul/vit_r50_l32_fe/1")

def vit_features(x):
    return vit_model(x)
# Load the model
model = tf.keras.models.load_model(
    MODEL_FILE,
    custom_objects={'KerasLayer': hub.KerasLayer,'vit_features': vit_features}  # Needed because you used TF-Hub
)

model.summary()

import numpy as np
from tensorflow.keras.preprocessing import image

IMG_SIZE = (224, 224)

def predict_image(img_path, model, class_indices):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    preds = model.predict(img_array)
    pred_class = np.argmax(preds, axis=1)[0]

    inv_class_map = {v: k for k, v in class_indices.items()}
    return inv_class_map[pred_class]

predict_image("1.jpg",model,{'Apple': 0, 'Bell Pepper': 1, 'Cherry': 2, 'Corn (Maize)': 3, 'Grape': 4, 'Peach': 5, 'Potato': 6, 'Strawberry': 7, 'Tomato': 8})


Model: "plant_id_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_image (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vit_feature_extractor (Lambda)  │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       262,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 892,445 (3.40 MB)

 Trainable params: 297,225 (1.13 MB)

 Non-trainable params: 768 (3.00 KB)

 Optimizer params: 594,452 (2.27 MB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 50s 50s/step


'Apple'

In [ ]:
predictor = PlantDiseasePredictor("models")  # Path to your saved models

# Predict from file path
result = predictor.predict_complete("image.jpg")